## <u style="color: blue">7. Gridsearch</u> 

Comme précisé dans le notebook principal, vous trouverez ici les calculs de Gridsearch pour les modèles suivants :
- Decision Tree
- Random Forest
- Linear Regression
- XGBoost
- SVM

In [1]:
# importation de toutes les librairies necessaires pour le gridsearch
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.linear_model import LogisticRegression
from Decision_Tree import DecisionTree
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
import xgboost as xgb

In [2]:
# import du csv
df_train = pd.read_csv('./CSV/train_tweets_cleaned.csv')

In [3]:
df_train.head()

,id,text,target,transformed_text,stemmed_text,vectorized_text
0,1,Deeds Reason earthquake ALLAH Forgive,1,"['deeds', 'reason', 'earthquake', 'allah', 'fo...","['deed', 'reason', 'earthquak', 'allah', 'forg...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,4,Forest fire near La Ronge Sask Canada,1,"['forest', 'fire', 'near', 'la', 'ronge', 'sas...","['forest', 'fire', 'near', 'la', 'rong', 'sask...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,5,residents asked shelter place notified officer...,1,"['residents', 'asked', 'shelter', 'place', 'no...","['resid', 'ask', 'shelter', 'place', 'notifi',...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,6,receive wildfires evacuation orders California,1,"['receive', 'wildfires', 'evacuation', 'orders...","['receiv', 'wildfir', 'evacu', 'order', 'calif...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,7,sent photo Ruby Alaska smoke wildfires pours s...,1,"['sent', 'photo', 'ruby', 'alaska', 'smoke', '...","['sent', 'photo', 'rubi', 'alaska', 'smoke', '...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [19]:
X = df_train['vectorized_text']
y = df_train['target']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y)

# XGBoost
# SVM
# Logistic Regression

Gridsearch pour le decision tree :

In [26]:
models_params = [
{
    'model': DecisionTreeClassifier(),
    params = {
    'max_depth': [3, 5, 10, 15],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'criterion': ['gini', 'entropy']
    },
},
{
    model: RandomForestClassifier(),
    params = {
    'n_estimators': [100, 200, 300, 400],
    'max_depth': [5, 10, 15, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 3, 4, 5],
    'criterion': ['gini', 'entropy']
    },
}
]
decision_tree_sklearn = DecisionTreeClassifier()

grid_search_decision_tree = GridSearchCV(decision_tree_sklearn, param_grid=params, cv=5, n_jobs=-1, scoring='accuracy')

In [27]:
grid_search_decision_tree.fit(X_train, y_train)

ValueError: 
All the 600 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\x_mat\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\x_mat\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\x_mat\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\tree\_classes.py", line 1009, in fit
    super()._fit(
  File "c:\Users\x_mat\AppData\Local\Programs\Python\Python312\Lib\site-packages

In [ ]:
print('Decision Tree best score :', grid_search_decision_tree.best_score_)
print('Decision Tree best parameters :', grid_search_decision_tree.best_params_)

Maintenant que nous en avons terminé avec la partie du Gridsearch, nous pouvons retourner sur le premier notebook afin de mettre en application les résultats obtenus.

## <u style="color: blue">[8. Evaluation des modèles](./Eye_Of_Emergency.ipynb#8.-Evaluation-des-modèles)</u>